In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

In [ ]:
!pip install -r requirements.txt

In [1]:
import sys
import platform
print("Python:", sys.version)
print("Platform:", platform.platform())

import tensorflow as tf
print("TensorFlow:", tf.__version__)

import numpy as np
print("NumPy:", np.__version__)

import torch
print("PyTorch:", torch.__version__)

from deepface import DeepFace
print("DeepFace importado com sucesso")


Python: 3.8.1 (tags/v3.8.1:1b293b6, Dec 18 2019, 23:11:46) [MSC v.1916 64 bit (AMD64)]
Platform: Windows-10-10.0.22621-SP0
TensorFlow: 2.13.0
NumPy: 1.24.3
PyTorch: 2.4.1+cpu
DeepFace importado com sucesso


In [8]:
import pathlib
import sys
import torch

# Corrigir paths no Windows
if sys.platform == "win32":
    pathlib.PosixPath = pathlib.WindowsPath

# Carregar modelo do YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'custom', path='modelo-1000img-40epochs.pt', source='github')
print("Modelo YOLOv5 carregado com sucesso")


Using cache found in C:\Users\jvalb/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-5-21 Python-3.8.1 torch-2.4.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Modelo YOLOv5 carregado com sucesso


In [11]:
import cv2
import sys
import os
import torch
from deepface import DeepFace

# Carregar modelo YOLOv5 personalizado
#model = torch.hub.load('ultralytics/yolov5', 'custom', path='modelo-800img-50epochs.pt', source='github', force_reload=True)

# Classe genérica se YOLO estiver treinado para múltiplos rótulos
class_names = {
    0: 'Pessoa',
    1: 'Joao',
    2: 'Gabriel'
}

# Abertura da webcam
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("Erro ao abrir a webcam")
    sys.exit()

frame_count = 0
emotion_cache = {}
face_cache = {}

print("Pressione 'q' para sair.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar frame")
        break

    results = model(frame)
    detections = results.xyxy[0].tolist()

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
        cls = int(cls)
        nome_detectado = class_names.get(cls, "Pessoa")

        # Recorta rosto detectado
        cropped_face = frame[y1:y2, x1:x2]

        # Cria imagem temporária para análise
        temp_path = "temp_face.jpg"
        cv2.imwrite(temp_path, cropped_face)

        # Reconhecimento facial (apenas a cada 15 frames ou novo rosto)
        if frame_count % 5 == 0 or cls not in face_cache:
            try:
                # Detecta baseado nos rostos da pasta faces_db
                df = DeepFace.find(img_path=temp_path, db_path="faces_db", enforce_detection=False)
                if len(df) > 0 and len(df[0]) > 0:
                    identity_path = df[0].iloc[0]['identity']
                    label_face = os.path.basename(identity_path).replace(".jpg", "").replace(".jpeg", "")
                else:
                    label_face = "Desconhecido"
                face_cache[cls] = label_face
            except Exception as e:
                face_cache[cls] = "Erro"

        label_face = face_cache.get(cls, "Desconhecido")

        # Análise de emoção — pode ser aplicada, mas aqui optamos por não utilizar
        # if frame_count % 15 == 0 or cls not in emotion_cache:
        #     try:
        #         analysis = DeepFace.analyze(
        #             img_path=cropped_face,
        #             actions=['emotion'],
        #             enforce_detection=False
        #         )[0]
        #         #emotion = analysis['dominant_emotion']
        #     except:
        #         emotion = "Desconhecido"
        #     emotion_cache[cls] = emotion

        # emotion = emotion_cache.get(cls, "Desconhecido")

        # Mostra rótulo com nome e emoção
        # final_label = f"{label_face} ({conf*100:.1f}%) - {emotion}"

        
        #final_label = f"{label_face} ({conf*100:.1f}%)"
        final_label = f"{label_face}"
        if final_label.lower().startswith("gabriel"):
            color = (3, 252, 177)
        elif final_label.lower().startswith("joao"):
            color = (255, 0, 0)
        else:
            color = (0, 0, 255)
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, final_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.imshow("Reconhecimento Facial + Emoção", frame)

    frame_count += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Pressione 'q' para sair.
25-05-23 09:44:01 - Searching temp_face.jpg in 4 length datastore
25-05-23 09:44:01 - find function duration 0.19217848777770996 seconds
25-05-23 09:44:01 - Searching temp_face.jpg in 4 length datastore
25-05-23 09:44:01 - find function duration 0.184281587600708 seconds
25-05-23 09:44:01 - Searching temp_face.jpg in 4 length datastore
25-05-23 09:44:02 - find function duration 0.1998758316040039 seconds
25-05-23 09:44:02 - Searching temp_face.jpg in 4 length datastore
25-05-23 09:44:02 - find function duration 0.19938921928405762 seconds
25-05-23 09:44:03 - Searching temp_face.jpg in 4 length datastore
25-05-23 09:44:03 - find function duration 0.1831836700439453 seconds
25-05-23 09:44:03 - Searching temp_face.jpg in 4 length datastore
25-05-23 09:44:03 - find function duration 0.19901752471923828 seconds
25-05-23 09:44:04 - Searching temp_face.jpg in 4 length datastore
25-05-23 09:44:04 - find function duration 0.18864798545837402 seconds
25-05-23 09:44:04 - 